In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import itertools as it
from MetricMethod import *
from phik import phik_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
df = pd.read_csv("diabetes_prediction_dataset.csv")
df.head()

In [ ]:
df.info()

In [ ]:
for coulumn in it.filterfalse(lambda x: x in 
                            set(['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes']),
                            it.islice(df.columns, 0, None)):
    mean = df[coulumn].mean()
    std = df[coulumn].std()

    lower_limit = mean - 3 * std
    upper_limit = mean + 3 * std

    ind = (df[coulumn] < lower_limit) | (df[coulumn] > upper_limit)
    out = df[ind][[coulumn, 'diabetes']]
    if len(out) > 0:
        print(f'Нижняя граница {lower_limit}')
        print(f'Верхняя граница {upper_limit}')
        print(f'Количество {len(out)}')
        print(out)

In [ ]:
for coulumn in it.filterfalse(lambda x: x in 
                            set(['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes']),
                            it.islice(df.columns, 0, None)):
    mean = df[coulumn].mean()
    std = df[coulumn].std()

    lower_limit = mean - 3 * std
    upper_limit = mean + 3 * std

    ind = (df[coulumn] < lower_limit) | (df[coulumn] > upper_limit)
    ind_low = (df[coulumn] < lower_limit)
    ind_high = (df[coulumn] > upper_limit)
    out_low = df[ind_low][[coulumn, 'diabetes']]
    out_high = df[ind_high][[coulumn, 'diabetes']]
    out = df[ind][[coulumn, 'diabetes']]
    if len(out) > 0:
        print(out.groupby('diabetes')[coulumn].count())
    if len(out_low) > 0:
        print(out_low.groupby('diabetes')[coulumn].count())   
    if len(out_high) > 0:
        print(out_high.groupby('diabetes')[coulumn].count())
    print("____________________________")

In [ ]:
df['gender'].value_counts()


In [ ]:

df.hist(figsize=(9, 12), xrot=15)

In [ ]:
t = df.drop(columns=['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes'])

# Получение количества колонок
num_columns = len(t.columns)

# Расчет количества строк и столбцов для сетки
num_rows = int((num_columns + 1) / 2)  # Округляем вверх до ближайшего целого числа

# Создание сетки подграфиков
fig, axes = plt.subplots(num_rows, 2, figsize=(8, 10))

# Распаковка двумерного массива осей в одномерный массив
axes = axes.flatten()

# Построение ящиковых диаграмм для каждой колонки
for i, column in enumerate(t.columns):
    # Построение ящиковой диаграммы для текущей колонки
    axes[i].boxplot(t[column].astype(float))
    axes[i].grid(True)
    
    # Установка заголовка для текущего графика
    axes[i].set_title(column)

# Удаление пустых подграфиков
if num_columns % 2 != 0:
    fig.delaxes(axes[-1])

# Расположение подграфиков
fig.tight_layout()

# Отображение графиков
plt.show()

In [ ]:
df.corr().round(4)

In [ ]:
phik_matrix(df).round(4)

In [ ]:
t = df[['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes']]

# Получение количества колонок
num_columns = len(t.columns)

# Расчет количества строк и столбцов для сетки
num_rows = int((num_columns + 1) / 2)  # Округляем вверх до ближайшего целого числа

# Создание сетки подграфиков
fig, axes = plt.subplots(num_rows, 2, figsize=(8, 10))

# Распаковка двумерного массива осей в одномерный массив
axes = axes.flatten()

# Построение ящиковых диаграмм для каждой колонки
for i, column in enumerate(t.columns):
    # Построение ящиковой диаграммы для текущей колонки
    #axes[i].plot(t[column], kind='bar')
    t[column].value_counts().plot(kind='bar', ax=axes[i])
    axes[i].grid(True)
    
    # Установка заголовка для текущего графика
    axes[i].set_title(column)

# Удаление пустых подграфиков
if num_columns % 2 != 0:
    fig.delaxes(axes[-1])

# Расположение подграфиков
fig.tight_layout()

# Отображение графиков
plt.show()

In [ ]:
df_processed = pd.get_dummies(df, columns=['smoking_history', 'gender'], 
                              drop_first=True)
df_processed

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.MinMaxScaler()

names = df_processed.columns
d = scaler.fit_transform(df_processed)

df_processed = pd.DataFrame(d, columns=names)
print(len(df_processed))
df_processed.head()

In [ ]:
# Разделите датафрейм на тренировочный и остаточный набор данных
train_df, temp_df = train_test_split(df_processed, test_size=0.3, random_state=42)

train_X = train_df.drop(columns='diabetes')
train_y = train_df['diabetes']

# Разделите остаточный набор данных на валидационный и тестовый наборы данных
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

val_X = val_df.drop(columns='diabetes')
val_y = val_df['diabetes']

test_X = test_df.drop(columns='diabetes')
test_y = test_df['diabetes']

# Выведите размеры полученных наборов данных
print("Размер тренировочного набора данных:", train_df.shape)
print("Размер валидационного набора данных:", val_df.shape)
print("Размер тестового набора данных:", test_df.shape)

In [ ]:
from imblearn.under_sampling import NearMiss

nm = NearMiss(version=1)
train_X_nm, train_y_nm = nm.fit_resample(train_X, train_y)
len(train_X_nm)

In [ ]:
from imblearn.under_sampling import NearMiss

nm = NearMiss(version=1)
train_X_nm, train_y_nm = nm.fit_resample(train_X, train_y)
len(train_X_nm)

In [ ]:
model = KNeighborsClassifier(n_neighbors=100)
model.fit(train_X_nm, train_y_nm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = KNN(countNeigbor=100, method='kdtree')
model.fit(train_X_nm, train_y_nm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = KNN_weight(countNeigbor=100, method='kdtree', weight_func= lambda x : 1/(x+0.00001))
model.fit(train_X_nm, train_y_nm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = ParzenWindowFixedWidth(width=0.15, method='kdtree')
model.fit(train_X_nm, train_y_nm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
model = ParzenWindowVariableWidth(countNeighbor=100, method='kdtree')
model.fit(train_X_nm, train_y_nm)
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')

In [ ]:
pred_y = np.array([0 for _ in range(len(test_y))])
print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')

In [ ]:
model = PotentialFunction(width=0.01, eps=0.05, method='kdtree')
model.fit(train_X_nm.sample(n=1000), train_y_nm.sample(n=1000))

In [ ]:
pred_y = model.predict(test_X)

print(f'Accuracy {round(accuracy_score(test_y, pred_y), 4)}')
print(f'F1 {round(f1_score(test_y, pred_y), 4)}')
print(f'Precision {round(precision_score(test_y, pred_y), 4)}')
print(f'Recall {round(recall_score(test_y, pred_y), 4)}')